<a href="https://colab.research.google.com/github/mamuncseru/analysis_on_epileptic_seizure_bonn_dataset/blob/main/Epileptic_Seizure_Prediction_using_EEG_bonn_data_with_BiLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import libraries

In [2]:
import sys
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import confusion_matrix

from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.utils.data as Data
from torch.optim import Adam, SGD

import keras
from keras.layers import Dense, Convolution1D, MaxPool1D, Flatten, Dropout
from keras.layers import Input, LSTM
from keras.layers import BatchNormalization
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint


# from tensorflow import keras
from tensorflow.keras import layers


## Reading Data

In [4]:
raw_data = pd.read_csv('https://raw.githubusercontent.com/mamuncseru/analysis_on_epileptic_seizure_bonn_dataset/main/Epileptic%20Seizure%20Recognition.csv')
raw_data.head()

,Unnamed,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X170,X171,X172,X173,X174,X175,X176,X177,X178,y
0,X21.V1.791,135,190,229,223,192,125,55,-9,-33,...,-17,-15,-31,-77,-103,-127,-116,-83,-51,4
1,X15.V1.924,386,382,356,331,320,315,307,272,244,...,164,150,146,152,157,156,154,143,129,1
2,X8.V1.1,-32,-39,-47,-37,-32,-36,-57,-73,-85,...,57,64,48,19,-12,-30,-35,-35,-36,5
3,X16.V1.60,-105,-101,-96,-92,-89,-95,-102,-100,-87,...,-82,-81,-80,-77,-85,-77,-72,-69,-65,5
4,X20.V1.54,-9,-65,-98,-102,-78,-48,-16,0,-21,...,4,2,-12,-32,-41,-65,-83,-89,-73,5


In [5]:
raw_data.shape

(11500, 180)

In [6]:
raw_data

,Unnamed,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X170,X171,X172,X173,X174,X175,X176,X177,X178,y
0,X21.V1.791,135,190,229,223,192,125,55,-9,-33,...,-17,-15,-31,-77,-103,-127,-116,-83,-51,4
1,X15.V1.924,386,382,356,331,320,315,307,272,244,...,164,150,146,152,157,156,154,143,129,1
2,X8.V1.1,-32,-39,-47,-37,-32,-36,-57,-73,-85,...,57,64,48,19,-12,-30,-35,-35,-36,5
3,X16.V1.60,-105,-101,-96,-92,-89,-95,-102,-100,-87,...,-82,-81,-80,-77,-85,-77,-72,-69,-65,5
4,X20.V1.54,-9,-65,-98,-102,-78,-48,-16,0,-21,...,4,2,-12,-32,-41,-65,-83,-89,-73,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11495,X22.V1.114,-22,-22,-23,-26,-36,-42,-45,-42,-45,...,15,16,12,5,-1,-18,-37,-47,-48,2
11496,X19.V1.354,-47,-11,28,77,141,211,246,240,193,...,-65,-33,-7,14,27,48,77,117,170,1
11497,X8.V1.28,14,6,-13,-16,10,26,27,-9,4,...,-65,-48,-61,-62,-67,-30,-2,-1,-8,5
11498,X10.V1.932,-40,-25,-9,-12,-2,12,7,19,22,...,121,135,148,143,116,86,68,59,55,3


In [15]:
data = raw_data.values
data = data[0:11501, 1:180]
data.shape

(11500, 179)

In [16]:
type(data)

numpy.ndarray

## Classifying data categories

In [17]:
D = data
df_1 = D[D[:, 178] == 1]
df_2 = D[D[:, 178] == 2]
df_3 = D[D[:, 178] == 3]
df_4 = D[D[:, 178] == 4]
df_5 = D[D[:, 178] == 5]

print(df_1.shape)
print(df_2.shape)
print(df_3.shape)
print(df_4.shape)
print(df_5.shape)

df_1 = df_1.astype(int)
df_2 = df_2.astype(int)
df_3 = df_3.astype(int)
df_4 = df_4.astype(int)
df_5 = df_5.astype(int)

(2300, 179)
(2300, 179)
(2300, 179)
(2300, 179)
(2300, 179)


## Experiment 1: Comparing CNN and BiLSTM on Epilepsy versus Healthy Data Categories

In [18]:
df_3[:, 178] = df_3[:, 178] - 3
D1 = np.concatenate([df_1, df_3])

In [21]:
D1.shape

(4600, 179)

### Creating training (80), validation(10) and test(10) data from category 1 and 3


In [20]:
number_of_rows = D1.shape[0]

random_indices = np.random.choice(number_of_rows, size=int(number_of_rows*0.8), replace=False)

label_train = D1[random_indices, -1]
data_train = D1[random_indices, :-1]

D1_rest = np.delete(D1, random_indices, 0)

number_of_rows = D1_rest.shape[0]
random_indices = np.random.choice(number_of_rows, size=int(number_of_rows*0.5), replace=False)

label_val = D1_rest[random_indices, -1]
data_val = D1_rest[random_indices, :-1]

D1_rest_rest = np.delete(D1_rest, random_indices, 0)

label_test = D1_rest_rest[:, -1]
data_test = D1_rest_rest[:, :-1]

data_train = np.expand_dims(data_train, axis=2)
data_val = np.expand_dims(data_val, axis=2)
data_test = np.expand_dims(data_test, axis=2)

print(label_train.shape, data_train.shape)
print(label_val.shape, data_val.shape)
print(label_test.shape, data_test.shape)

(3680,) (3680, 178, 1)
(460,) (460, 178, 1)
(460,) (460, 178, 1)


### Define model evaluation function

In [23]:
def evaluate_model(history, X_test, y_test, model):
  scores = model.evaluate((X_test), y_test, verbose=0)
  print("Accuracy: %.2f%%" % (scores[1]*100))

  print(history)
  fig1, ax_acc = plt.subplots()
  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])
  plt.xlabel('Epoch')
  plt.ylabel('Accuracy')
  plt.title('Model - Accuracy')
  plt.legend(['Training', 'Validation'], loc='lower right')
  plt.show()

  fig2, ax_loss = plt.subplots()
  plt.xlabel('Epoch')
  plt.ylabel('Loss')
  plt.title('Model - Loss')
  plt.legend(['Training', 'Validation'], loc='upper right')
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.show()
  target_names = ['1', '2', '3']

  y_true = []
  for element in y_test:
    y_true.append(np.argmax(element))

  prediction_proba = model.predict(X_test)
  prediction = np.argmax(prediction_proba, axis=1)
  cnf_matrix = confusion_matrix(y_true, prediction)

### Define CNN network

In [24]:
def network_CNN(X_train, y_train):
  im_shape = (X_train.shape[1], 1)
  inputs_cnn = Input(shape=(im_shape), name='inputs_cnn')

  conv1d_1 = layers.Conv1D(filters=32, kernel_size=6)(inputs_cnn)
  batch_normalization = BatchNormalization()(conv1d_1)
  max_pooling1d = layers.MaxPooling1D(2, padding='same')(batch_normalization)

  conv1d_2 = layers.Conv1D(filters=64, kernel_size=3)(max_pooling1d)
  batch_normalization_1 = BatchNormalization()(conv1d_2)
  max_pooling1d_1 = layers.MaxPooling1D(2, padding='same')(batch_normalization_1)

  flatten = Flatten()(max_pooling1d_1)

  dense = Dense(32)(flatten)
  dense_1 = Dense(16)(dense)

  main_output = Dense(2)(dense_1)

  model1 = Model(inputs = inputs_cnn, outputs=main_output)
  model1.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

  return model1


In [25]:
## Define CNN model to be trained on epileptic vs healthy data
model1 = network_CNN(data_train, label_train)
print(model1.summary())

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs_cnn (InputLayer)     [(None, 178, 1)]          0         
                                                                 
 conv1d (Conv1D)             (None, 173, 32)           224       
                                                                 
 batch_normalization (BatchN  (None, 173, 32)          128       
 ormalization)                                                   
                                                                 
 max_pooling1d (MaxPooling1D  (None, 87, 32)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 85, 64)            6208      
                                                                 
 batch_normalization_1 (Batc  (None, 85, 64)           256   

In [ ]:
from sqlalchemy.sql.expression import true
## Train CNN network on epileptic vs healthy data
# Save model at highest validation accuracy
save_path = 'checkpoint_1'
model_checkpoint_callback = keras.callbacks.ModelCheckpoint(
    filepath = save_path,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True
)

history = model1.fit(data_train, label_train, epochs=500, batch_size=32, validation_data=(data_val, label_val), callbacks=[model_checkpoint_callback])